In [1]:
from src.BOVW import BOVW
from src.ColorHistogram import ColorHistogram
from src.DeepModel import DeepModel
from src.CombinedModel import CombinedModel
from src.CNN import CNN
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
import dill
from tqdm import tqdm
from time import perf_counter
import os
from src.Preprocess import find_face_and_preprocess
from mtcnn import MTCNN

## Split new features (only if needed)

In [2]:
#feature_path = "./src/CNN_transfer/cnn_withus_maxpool/cnn_train_features.pkl"
#labels_path = "./src/CNN_transfer/cnn_withus_maxpool/cnn_train_labels.pkl"
#
#with open(feature_path, 'rb') as f: feature = dill.load(f)
#with open(labels_path, 'rb') as f: labels = dill.load(f)
#
#feature_savory = [feature[i] for i in np.where(np.asarray(labels)=='savory')[0]]
#feature_unsavory = [feature[i] for i in np.where(np.asarray(labels)=='unsavory')[0]]
#
#with open("./src/CNN_transfer/cnn_withus_maxpool/feature_savory.pkl", 'wb') as f: feature = dill.dump(feature_savory, f)
#with open("./src/CNN_transfer/cnn_withus_maxpool/feature_unsavory.pkl", 'wb') as f: feature = dill.dump(feature_unsavory, f)

## Loading models

In [2]:
# BOVW paths
new_bovw_path = './src/BOVW/bovw_withus/bovw.pkl'
new_bovw_features = "./src/BOVW/bovw_withus/train_bovw.pkl"
new_bovw_savory = './src/BOVW/bovw_withus/feature_savory.pkl'
new_bovw_unsavory = './src/BOVW/bovw_withus/feature_unsavory.pkl'
new_train_image_path_bovw = './src/BOVW/bovw_withus/train_paths.pkl'
# Color paths
new_color_path = "./src/Color/color_withus/histogram_model.pkl"
new_color_features = "./src/Color/color_withus/train_color_histogram.pkl"
new_color_savory = "./src/Color/color_withus/feature_savory.pkl"
new_color_unsavory = "./src/Color/color_withus/feature_unsavory.pkl"
new_train_image_path_color = './src/Color/color_withus/train_paths.pkl'
# Combined paths
new_combined_path = "./src/Combined_descriptors/combined_withus/combined_model.pkl"
# Siamese model paths
classifier_path = "./src/CNN_training/Classifier_withus/svm_final_nu03.sav"
siamese_embeddings = "./src/CNN_training/final_siamese_embedding_model_withus"
siamese_features = "./src/CNN_training/Features_withus/feature_final.pkl"
siamese_savory = "./src/CNN_training/Features_withus/feature_savory.pkl"
siamese_unsavory = "./src/CNN_training/Features_withus/feature_unsavory.pkl"
image_train_paths = "./src/CNN_training/Features_withus/path_final.pkl"
# CNN
cnn_path = './src/CNN/trained_cnn/'
cnn_features = "./src/CNN/cnn_withus_maxpool/cnn_train_features.pkl"
cnn_savory = './src/CNN/cnn_withus/feature_savory.pkl'
cnn_unsavory = './src/CNN/cnn_withus/feature_unsavory.pkl'
cnn_image_train_paths = "./src/CNN/cnn_withus/cnn_train_paths.pkl"

In [3]:
detector = MTCNN()

In [4]:
model = DeepModel(
        classifier_path, 
        siamese_embeddings, 
        siamese_features,
        image_train_paths, 
        siamese_savory,
        siamese_unsavory, 
        (200, 200)
        )
color_model = ColorHistogram.load_model(new_color_path)
bovw_model = BOVW.load_model(new_bovw_path)
combined_model = CombinedModel.load_model(new_combined_path)
cnn_model = CNN(cnn_path)

Model loaded from src\CNN_training\final_siamese_embedding_model_withus
KDTree computed in: 0.699861300000002


## Evaluate CBIR performances on new image of ourselves (live version)

In [5]:
import cv2
from cv2 import VideoCapture
from PIL import Image
import tempfile

In [6]:
def imageSubplot(similar_s, similar_u, dist_s, dist_u):
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 10))
    fig.suptitle('** I row: Savory - II row Unsavory **', fontsize=15)
    idx = np.argsort(np.argsort(np.concatenate((dist_s[0], dist_u[0]))))
    for i, path in enumerate(similar_s):
        plt.subplot(2, 5, i+1)
        plt.title(f"/{i+1} - Rank: {idx[i]+1} - dist: {np.round(dist_s[0][i], decimals=3)}")
        plt.axis('off')
        plt.imshow(plt.imread(path))
    for i, path in enumerate(similar_u):
        plt.subplot(2, 5, i+6)
        plt.title(f"/{i+6} - Rank: {idx[i+5]+1}- dist: {np.round(dist_u[0][i], decimals=3)}")
        plt.axis('off')
        plt.imshow(plt.imread(path))
    #plt.savefig("./cbir.jpg", bbox_inches='tight')
    plt.show()
    
def imageSubplot_perfo(preds, preds_score, similar_s, similar_u, dist_s, dist_u):
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 10))
    if preds =='0':
        preds = "savory"
    elif preds =='1':
        preds = "unsavory"
    confidence = np.round(np.max(preds_score), decimals=2)*100 if preds_score is not None else "..."
    fig.suptitle(f'* You are: {preds}, with {confidence} confidence! *', fontsize=15)
    idx = np.argsort(np.argsort(np.concatenate((dist_s, dist_u))))
    for i, path in enumerate(similar_s):
        label = similar_s[i].split("\\")[-2]
        plt.subplot(2, 5, i+1)
        plt.title(f"/{i+1} - {label} - Rank: {idx[i]+1} - dist: {np.round(dist_s[i], decimals=3)}")
        plt.axis('off')
        plt.imshow(plt.imread(path))
    for i, path in enumerate(similar_u):
        label = similar_u[i].split("\\")[-2]
        plt.subplot(2, 5, i+6)
        plt.title(f"/{i+6} - {label} - Rank: {idx[i+5]+1} - dist: {np.round(dist_u[i], decimals=3)}")
        plt.axis('off')
        plt.imshow(plt.imread(path))
    plt.show()

In [6]:
def start_demo():  
    try:
        cam = VideoCapture(0)
        print("recording...")
        while True and cam.isOpened():
            s, frame = cam.read()   # frame is a numpy.ndarray
            cv2.imshow('frame', frame)
            if not s:
                break
            if cv2.waitKey(20) & 0xFF == ord('q'):
                img = Image.fromarray(frame).convert('RGB')
                r, g, b = img.split()
                result = Image.merge('RGB', (b, g, r))  # To resolve blueish image
                result.save("./temp.jpg")
                faces = find_face_and_preprocess("./temp.jpg", detector)
                if len(faces) >= 1:
                    for i, f in enumerate(faces):
                        f = (f*255).astype('uint8')
                        image = Image.fromarray(f)
                        image.save("./temp_crop.jpg")
                        result, most_similar_s, most_similar_u, feature, dist_s, dist_u = BOVW.cbir(new_bovw_path, "./temp_crop.jpg", new_bovw_savory, new_bovw_unsavory, new_train_image_path_bovw)
                        print(result[0])
                        # CBIR
                        imageSubplot(most_similar_s, most_similar_u, dist_s, dist_u)
                else: print("faccia non trovata...")
                break
        cam.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print("error...", e)
        cam.release()
        cv2.destroyAllWindows()

In [ ]:
# Press q to get image from recorder
start_demo()

### Offline version

In [ ]:
img_path = "C:/Users/Jarvis/Desktop/io.jpg"

temp_dir = tempfile.TemporaryDirectory()

faces = find_face_and_preprocess(img_path, detector)
if len(faces) >= 1:
    for i, f in enumerate(faces):
        f = (f*255).astype('uint8')
        image = Image.fromarray(f)
        image.save(temp_dir.name+"./temp_crop.jpg")
        #pred_score, result, most_similar, feature, dist = ColorHistogram.cbir_performance(new_color_path, temp_dir.name+"./temp_crop.jpg", new_color_features, new_train_image_path_color, k=10)                        
        #pred_score, result, most_similar, feature, dist = BOVW.cbir_performance(new_bovw_path, temp_dir.name+"./temp_crop.jpg", new_bovw_features, new_train_image_path_color, k=10)                        
        #pred_score, result, most_similar, feature, dist = CombinedModel.cbir_performance(new_combined_path, temp_dir.name+"./temp_crop.jpg", None, new_train_image_path_color, k=10)                        
        #pred_score, result, most_similar, feature, dist = cnn_model.cbir_performance(temp_dir.name+"./temp_crop.jpg", cnn_features, cnn_image_train_paths, k=10)                        
        pred_score, result, most_similar, feature, dist = model.cbir_performance(image, k=10)                        
        # CBIR
        len_similar = len(most_similar)
        len_dists = len(dist[0])
        most_similar_s = most_similar[:len_similar//2]
        most_similar_u = most_similar[len_similar//2:]
        dist_s = dist[0][:len_dists//2]
        dist_u = dist[0][len_dists//2:]
        imageSubplot_perfo(result[0], pred_score, most_similar_s, most_similar_u, dist_s, dist_u)
else: print("faccia non trovata...")


# Evaluate CBIR performance

In [ ]:
%%capture
query_dataset = "../dataset/query_cbir/"
paths = list(Path(query_dataset).rglob("*.jpg"))

temp_dir = tempfile.TemporaryDirectory()

most_similar_color, most_similar_bovw, most_similar_comb, most_similar_cnn, most_similar_siamese = [], [], [], [], []
gt = []

for image_path in paths:
    gt.append(image_path.parts[-2])
    print(image_path)
    faces = find_face_and_preprocess(image_path, detector)
    if faces is not None and len(faces) >= 1:
        for i, f in enumerate(faces):
            f = (f*255).astype('uint8')
            image = Image.fromarray(f)
            image.save(temp_dir.name+"./temp_crop.jpg")
            _, _, similar_color, _, _ = ColorHistogram.cbir_performance(new_color_path, temp_dir.name+"./temp_crop.jpg", new_color_features, new_train_image_path_color, k=10)                        
            _, _, similar_bovw, _, _ = BOVW.cbir_performance(new_bovw_path, temp_dir.name+"./temp_crop.jpg", new_bovw_features, new_train_image_path_color, k=10)                        
            _, _, similar_comb, _, _ = CombinedModel.cbir_performance(new_combined_path, temp_dir.name+"./temp_crop.jpg", None, new_train_image_path_color, k=10)                        
            _, _, similar_cnn, _, _ = cnn_model.cbir_performance(temp_dir.name+"./temp_crop.jpg", cnn_features, cnn_image_train_paths, k=10)                        
            _, _, similar_siamese, _, _ = model.cbir_performance(image, k=10)                        
            most_similar_color.append(similar_color)
            most_similar_bovw.append(similar_bovw)
            most_similar_comb.append(similar_comb)
            most_similar_cnn.append(similar_cnn)
            most_similar_siamese.append(similar_siamese)            
    else: print("faccia non trovata...")

In [75]:
most_similar_color = [[i.split("\\")[-2] for i in x] for x in most_similar_color]
most_similar_bovw = [[i.split("\\")[-2] for i in x] for x in most_similar_bovw]
most_similar_comb = [[i.split("\\")[-2] for i in x] for x in most_similar_comb]
most_similar_cnn = [[i.split("\\")[-2] for i in x] for x in most_similar_cnn]
most_similar_siamese = [[i.split("\\")[-2] for i in x] for x in most_similar_siamese]

In [77]:
def average_precision(actual, predicted, k=10):
    score = 0.0
    num_hits = 0.0
    if not actual: return 0.0
    if k > len(predicted): return None
    for i,p in enumerate(predicted[:k]):
        if p == actual:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / k

In [166]:
models = [most_similar_color, most_similar_bovw, most_similar_comb, most_similar_cnn, most_similar_siamese]
models_name = ["Color Histogram", "BOVW", "BOVW+Color", "CNN", "Siamese"]

def get_mapk(models, models_name, K):
    aps = {}
    for y, m in enumerate(models):
        aps[models_name[y]] = []
        for i in range(len(gt)):
            for K in [K]:
                aps[models_name[y]].append(average_precision(gt[i], m[i], k=K))
        aps[models_name[y]] = np.mean(aps[models_name[y]])
    return aps

In [167]:
get_mapk(models, models_name, 1)

{'Color Histogram': 0.5,
 'BOVW': 0.6,
 'BOVW+Color': 0.55,
 'CNN': 0.75,
 'Siamese': 0.65}

In [163]:
get_mapk(models, models_name, 5)

{'Color Histogram': 0.45133333333333325,
 'BOVW': 0.4423333333333333,
 'BOVW+Color': 0.47733333333333333,
 'CNN': 0.6875,
 'Siamese': 0.46533333333333327}

In [161]:
get_mapk(models, models_name, 10)

{'Color Histogram': 0.44391865079365084,
 'BOVW': 0.436827380952381,
 'BOVW+Color': 0.43923015873015875,
 'CNN': 0.6559781746031745,
 'Siamese': 0.4347996031746031}